In [ ]:
import pandas as pd, numpy as np, matplotlib.pylab as plt
import MySQLdb

In [ ]:
# Open database connection
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

In [ ]:
user_visits = pd.read_sql('''
    SELECT * FROM qyw_7th_visit_mean;
    ''', con=conn)
user_visits.insert(0, 'CASE_ID','')

In [ ]:
# 执行5min左右
import datetime
unique_users = user_visits['USER_ID'].unique()
for user_id in unique_users:
    # 一定要选择mergesort，这样才是稳定的（VISIT_TIME相同的行的index保证原有顺序）
    single_user_visits = user_visits[user_visits['USER_ID'] == user_id].sort_values(by=['VISIT_TIME'],ascending=[True],kind='mergesort')
    # 一定要更新index，否则index+1不对（因为sort_values有可能调换各行的顺序）
    single_user_visits.index = range(0,len(single_user_visits.index))
    size = len(single_user_visits.index)
    start = single_user_visits.index[0]
    date = None
    cid = 0
    for index, row in single_user_visits.iterrows():
        start_operate_time = row['VISIT_TIME']
        cur_date = start_operate_time.strftime('%Y-%m-%d')
        if not date:
            date = cur_date
        end_operate_time = start_operate_time
        next_date = cur_date
        if index < start+size-1:
            end_operate_time = single_user_visits.loc[index+1, 'VISIT_TIME']
            next_date = end_operate_time.strftime('%Y-%m-%d')
        interval = end_operate_time - start_operate_time
        interval_secs = interval.total_seconds()
        case_id = str(row['USER_ID'])+'@'+date+'_'+str(cid)
        single_user_visits.ix[index, 'CASE_ID'] = case_id
        if interval_secs > 5*60:
            if date != next_date:
                cid = 0
                date = next_date
            else:
                cid += 1
    single_user_visits.to_sql('qyw_7th_event', conn, flavor='mysql', if_exists='append', index=False)

In [ ]:
user_visits

In [ ]:
single_user_visits